<a href="https://colab.research.google.com/github/pierrot73/GenAIBootCamp/blob/Bootcamp/Week5_Day4_DC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
!pip install spacy torch scikit-learn
!python -m spacy download en_core_web_sm

# Importation de la bibliothèque pandas pour la manipulation des données
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

# Lecture du fichier csv
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/stock_market_dataset.csv')
df.head()


import pandas as pd

# Vérifier s'il y a des valeurs manquantes dans tout le DataFrame
if df.isnull().values.any():
    print("Il y a des valeurs manquantes.")
else:
    print("Il n'y a pas de valeurs manquantes.")

import pandas as pd

# Si colonnes catégorielles, afficher leurs valeurs uniques
cat_cols = df.select_dtypes(include='object').columns
if len(cat_cols) > 0:
    for col in cat_cols:
        print(f"\n{col} :\n{df[col].value_counts()}")
else:
    print("Il n'y a pas de valeurs catégorielles")


#Test Statistics with skimpy
from skimpy import skim
skim(df)


import pandas as pd
from sklearn.preprocessing import MinMaxScaler

# 1. Supprimer les colonnes inutiles
# Selon votre description, les colonnes "date", "symbol" peuvent être conservées ou supprimées
# Si elles ne sont pas nécessaires pour la modélisation, on peut les supprimer
df = df.drop(['date', 'symbol'], axis=1)

# 2. Créer la colonne 'target' : le prix de clôture du jour suivant
# On décale la colonne 'close' d'une ligne vers le haut
df['target'] = df['close'].shift(-1)

# 3. Supprimer la dernière ligne qui aura un target NaN après décalage
df = df.dropna()

# 4. Normaliser les données
scaler = MinMaxScaler()
# On ajuste et transforme toutes les colonnes sauf 'target' si besoin
# Mais généralement, on normalise toutes les features, y compris 'close' et autres
# Si vous souhaitez normaliser toutes les colonnes :
features = df.columns
df_scaled = pd.DataFrame(scaler.fit_transform(df), columns=features)

# 5. Vérification
print(df_scaled.head())

# À partir de là, vous pouvez convertir votre DataFrame en tableau numpy pour le modèle LSTM
# par exemple :
import numpy as np

# Convertir en numpy array
data = df_scaled.values

# Si vous souhaitez créer des séquences pour LSTM :
def create_sequences(data, seq_length=50):
    X = []
    y = []
    for i in range(len(data) - seq_length):
        X.append(data[i:i+seq_length, :-1])  # toutes les features sauf target
        y.append(data[i+seq_length, -1])    # target (prix de clôture du lendemain)
    return np.array(X), np.array(y)

# Exemple avec une longueur de séquence de 50
X, y = create_sequences(data, seq_length=50)

# Vérification des dimensions
print("X shape:", X.shape)
print("y shape:", y.shape)


import torch
from torch.utils.data import Dataset, DataLoader, random_split

# X et y sont déjà préparés
# X : numpy array de séquences (nb_samples, sequence_length, nb_features)
# y : numpy array des cibles (nb_samples,)

# Convertir en tensors PyTorch
X_tensor = torch.tensor(X, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.float32)

# Définir la classe Dataset personnalisée
class StockDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

# Créer l'objet Dataset
dataset = StockDataset(X_tensor, y_tensor)

# Définir la taille des splits
total_size = len(dataset)
train_size = int(0.7 * total_size)       # 70% pour l'entraînement
val_size = int(0.15 * total_size)        # 15% pour la validation
test_size = total_size - train_size - val_size  # Reste pour le test

# Split le dataset en plusieurs
train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

# Créer les DataLoaders
batch_size = 64  # par exemple

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Vérification
for batch_X, batch_y in train_loader:
    print("Batch X shape:", batch_X.shape)  # (batch_size, seq_length, nb_features)
    print("Batch y shape:", batch_y.shape)  # (batch_size,)
    break  # Affiche le premier batch pour vérification
import torch
import torch.nn as nn

class LSTM_GRU_Model(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, dropout=0.2):
        super(LSTM_GRU_Model, self).__init__()

        # Première couche LSTM
        self.lstm = nn.LSTM(
            input_size=input_dim,     # Nombre de features en entrée
            hidden_size=hidden_dim,   # Nombre de neurones dans la couche LSTM
            num_layers=num_layers,    # Nombre de couches LSTM empilées
            batch_first=True,         # Format des données : (batch, seq_len, features)
            dropout=dropout if num_layers > 1 else 0  # Dropout entre couches si >1 couche
        )

        # Couche GRU
        self.gru = nn.GRU(
            input_size=hidden_dim,
            hidden_size=hidden_dim,
            num_layers=1,
            batch_first=True,
            dropout=0  # Pas de dropout ici, vous pouvez l'ajouter si besoin
        )

        # Dropout après les couches RNN
        self.dropout = nn.Dropout(dropout)

        # Couche Fully Connected (Dense)
        self.fc = nn.Linear(hidden_dim, 1)  # 1 sortie pour la prédiction continue

    def forward(self, x):
        # Passage dans la couche LSTM
        lstm_out, _ = self.lstm(x)
        # lstm_out : (batch, seq_len, hidden_dim)

        # Passer la dernière sortie de la séquence dans la GRU
        gru_out, _ = self.gru(lstm_out)
        # gru_out : (batch, seq_len, hidden_dim)

        # Prendre la dernière sortie de la séquence (pour la prédiction finale)
        final_hidden_state = gru_out[:, -1, :]  # (batch, hidden_dim)

        # Appliquer Dropout
        out = self.dropout(final_hidden_state)

        # Passer dans la couche Dense
        out = self.fc(out)

        return out.squeeze()  # Retourner un vecteur (batch,)

#nb_features correspond au nombre de colonnes features (sans target donc 7).
model = LSTM_GRU_Model(input_dim=7, hidden_dim=64, num_layers=2, dropout=0.3)
print(model)
print(LSTM_GRU_Model)
print((model.lstm))
print((model.gru))
print((model.dropout))
print((model.fc))
import torch
import torch.nn as nn
import torch.optim as optim

# Le modèle s'appelle 'model'
# et que vous avez déjà défini vos DataLoaders : train_loader, val_loader

# 1. Définir la fonction de perte (Loss Function)
criterion = nn.MSELoss()  # Pour une tâche de régression

# 2. Définir l'optimiseur
learning_rate = 0.001
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# 3. Fonction pour entraîner une époque
def train_epoch(model, loader, criterion, optimizer, device):
    model.train()
    total_loss = 0
    for batch_X, batch_y in loader:
        batch_X = batch_X.to(device)
        batch_y = batch_y.to(device)

        optimizer.zero_grad()  # Réinitialiser les gradients
        outputs = model(batch_X)  # Forward pass
        loss = criterion(outputs, batch_y)  # Calcul de la perte
        loss.backward()  # Backpropagation
        optimizer.step()  # Mise à jour des paramètres

        total_loss += loss.item() * batch_X.size(0)
    return total_loss / len(loader.dataset)

# 4. Fonction pour valider une époque
def validate_epoch(model, loader, criterion, device):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for batch_X, batch_y in loader:
            batch_X = batch_X.to(device)
            batch_y = batch_y.to(device)

            outputs = model(batch_X)
            loss = criterion(outputs, batch_y)
            total_loss += loss.item() * batch_X.size(0)
    return total_loss / len(loader.dataset)

# 5. Boucle d’entraînement principale
num_epochs = 50  # Nombre d’époques
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

for epoch in range(1, num_epochs + 1):
    train_loss = train_epoch(model, train_loader, criterion, optimizer, device)
    val_loss = validate_epoch(model, val_loader, criterion, device)

    print(f"Epoch {epoch}/{num_epochs} - "
          f"Training Loss: {train_loss:.4f} - Validation Loss: {val_loss:.4f}")

from sklearn.metrics import r2_score
import joblib

model.eval()
all_preds = []
all_targets = []

with torch.no_grad():
    for test_X, test_y in test_loader:
        test_X, test_y = test_X.to(device), test_y.to(device)
        preds = model(test_X)
        all_preds.append(preds.view(-1).cpu())       # Correction ici
        all_targets.append(test_y.view(-1).cpu())    # Et ici

# Concaténer les prédictions et les cibles
all_preds = torch.cat(all_preds).numpy()
all_targets = torch.cat(all_targets).numpy()

# Calcul du R²
r2 = r2_score(all_targets, all_preds)
print(f"📈 Score R² sur le jeu de test : {r2:.4f}")

# Sauvegarde du scaler
joblib.dump(scaler, "minmax_scaler.pkl")
print("✅ Scaler sauvegardé sous minmax_scaler.pkl")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 41.4 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Il n'y a pas de valeurs manquantes.

date :
date
5-4-2018      1
12-27-2021    1
12-26-2021    1
12-25-2021    1
12-24-2021    1
             ..
12-17-2021    1
12-18-2021    1
12-19-2021    1
12-20-2021    1
12-21-2021    1
Name: count, Length: 1334, dtype: int64

symbol :
symbol
XRP-USDT    1334
Name: count, dtype: int64


╭──────────────────────────────────────────────── skimpy summary ─────────────────────────────────────────────────╮
│          Data Summary                Data Types                                                                 │
│ ┏━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓ ┏━━━━━━━━━━━━━┳━━━━━━━┓                                                          │
│ ┃ Dataframe         ┃ Values ┃ ┃ Column Type ┃ Count ┃                                                          │
│ ┡━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩ ┡━━━━━━━━━━━━━╇━━━━━━━┩                                                          │
│ │ Number of rows    │ 1334   │ │ float64     │ 7     │                                                          │
│ │ Number of columns │ 9      │ │ string      │ 2     │                                                          │
│ └───────────────────┴────────┘ └─────────────┴───────┘                                                          │
│                                                     number                                                      │
│ ┏━━━━━━━━━━┳━━━━┳━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━┓  │
│ ┃ column   ┃ NA ┃ NA % ┃ mean     ┃ sd       ┃ p0       ┃ p25      ┃ p50      ┃ p75      ┃ p100     ┃ hist   ┃  │
│ ┡━━━━━━━━━━╇━━━━╇━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━┩  │
│ │ unix     │  0 │    0 │ 15830000 │ 33280000 │ 15250000 │ 15540000 │ 15830000 │ 16120000 │ 16410000 │ ██████ │  │
│ │          │    │      │    00000 │      000 │    00000 │    00000 │    00000 │    00000 │    00000 │        │  │
│ │ open     │  0 │    0 │   0.4733 │   0.3199 │   0.1354 │   0.2563 │    0.326 │   0.5652 │    1.834 │  █▃▁▁  │  │
│ │ high     │  0 │    0 │   0.4956 │   0.3412 │   0.1494 │   0.2636 │   0.3376 │   0.5999 │    1.967 │  █▃▁▁  │  │
│ │ low      │  0 │    0 │   0.4501 │   0.2975 │   0.1013 │     0.25 │   0.3155 │   0.5289 │    1.652 │  █▃▁▁  │  │
│ │ close    │  0 │    0 │   0.4736 │   0.3201 │   0.1355 │   0.2563 │    0.326 │   0.5662 │    1.835 │  █▃▁▁  │  │
│ │ Volume   │  0 │    0 │ 35670000 │ 59150000 │  2385000 │ 64490000 │ 14810000 │ 37800000 │ 86080000 │   █    │  │
│ │ XRP      │    │      │        0 │        0 │          │          │        0 │        0 │       00 │        │  │
│ │ Volume   │  0 │    0 │ 23560000 │ 48410000 │  2198000 │ 18940000 │ 41360000 │ 25190000 │ 45900000 │   █    │  │
│ │ USDT     │    │      │        0 │        0 │          │          │          │        0 │       00 │        │  │
│ └──────────┴────┴──────┴──────────┴──────────┴──────────┴──────────┴──────────┴──────────┴──────────┴────────┘  │
│                                                     string                                                      │
│ ┏━━━━━━━━┳━━━━┳━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┓  │
│ ┃        ┃    ┃      ┃          ┃            ┃          ┃          ┃ chars per   ┃ words per   ┃             ┃  │
│ ┃ column ┃ NA ┃ NA % ┃ shortest ┃ longest    ┃ min      ┃ max      ┃ row         ┃ row         ┃ total words ┃  │
│ ┡━━━━━━━━╇━━━━╇━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━┩  │
│ │ date   │  0 │    0 │ 9-9-2021 │ 12-27-2021 │ 1-1-2019 │ 9-9-2021 │        8.98 │           1 │        1334 │  │
│ │ symbol │  0 │    0 │ XRP-USDT │ XRP-USDT   │ XRP-USDT │ XRP-USDT │           8 │           1 │        1334 │  │
│ └────────┴────┴──────┴──────────┴────────────┴──────────┴──────────┴─────────────┴─────────────┴─────────────┘  │
╰────────────────────────────────────────────────────── End ──────────────────────────────────────────────────────╯

       unix      open      high       low     close  Volume XRP  Volume USDT  \
0  1.000000  0.461933  0.426033  0.527812  0.463227    0.000000     0.000000   
1  0.999305  0.464995  0.431370  0.518270  0.461696    0.018714     0.032203   
2  0.998523  0.456870  0.432251  0.513693  0.464757    0.028781     0.049704   
3  0.997741  0.505558  0.466143  0.512597  0.456694    0.065635     0.116724   
4  0.997046  0.481832  0.477202  0.538900  0.505305    0.055433     0.102607   

     target  
0  0.461696  
1  0.464757  
2  0.456694  
3  0.505305  
4  0.481529  
X shape: (1283, 50, 7)
y shape: (1283,)
Batch X shape: torch.Size([64, 50, 7])
Batch y shape: torch.Size([64])
LSTM_GRU_Model(
  (lstm): LSTM(7, 64, num_layers=2, batch_first=True, dropout=0.3)
  (gru): GRU(64, 64, batch_first=True)
  (dropout): Dropout(p=0.3, inplace=False)
  (fc): Linear(in_features=64, out_features=1, bias=True)
)
<class '__main__.LSTM_GRU_Model'>
LSTM(7, 64, num_layers=2, batch_first=True, dropout=0.3)
GRU(64, 